In [1]:
import os

# set env var
os.environ["OPENAI_API_KEY"] = "your-api-key"

In [2]:
from typing import List, Dict, Any, Optional, Literal

from langchain.pydantic_v1 import Field, BaseModel
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_openai_fn_runnable,
    create_structured_output_chain,
    create_structured_output_runnable,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [138]:
class Clause(BaseModel):
    """Unfair clause extracted from the input document"""
    text: str = Field(None, description="Literal text of the clause extracted from the input document")
    unfairness: int = Field(..., choices=[1, 2, 3], description="The level of the clause: 1 clearly fair, 2 potentially unfair, 3 clearly unfair")

class UnfairClauses(BaseModel):
    """List of unfair clauses extracted from the input document"""
    clauses: List[Clause] = Field(None, description="List of unfair clauses extracted from the input document")


In [139]:
j_system_prompt = """
You are a lawyer and you are reviewing online Terms of Service in search of Jurisdiction clauses.
Jurisdictions clauses stipulate what courts will have the competence to adjudicate disputes under the contract.
Follow this instructions carefully:
- Identify all jurisdiction clauses in the given ToS.
- Mark jurisdiction clauses giving consumers a right to bring disputes in their place of residence were marked as clearly fair (1).
- Mark as clauses stating that any judicial proceeding takes place in a diferent city, country, etc. as clearly unfair (3).
- Mark only the minimun amount of text necessary to understand the clause.
"""

law_system_prompt = """
You are a lawyer and you are reviewing online Terms of Service in search of Choice of Law clauses.
Choice of law This clause specifes what law will govern the contract, meaning also what law will be applied in potential adjudication of a dispute arising.
Follow this instructions carefully:
- Identify all choice of law clauses in the given ToS.
- Mark clauses defining the applicable law as the law of the consumer’s country of residence as clearly fair (1), everything else as potentially unfair (2).
- Mark only the minimun amount of text necessary to understand the clause.
"""

ltd_system_prompt = """
You are a lawyer and you are reviewing online Terms of Service in search of Limitation of Liability clauses (LTD).
Limitation of liability This clause stipulates that the duty to pay damages is limited or excluded, for certain kinds of losses and under certain conditions.
Follow this instructions carefully:
- Identify all choice of law clauses in the given ToS.
- Mark clauses that explicitly afirm non-excludable providers’ liabilities as clearly fair (1).
- Mark clauses that reduce, limit, or exclude the liability of the service provider for physical injuries, intentional damages as well as in case of gross negligence as clearly unfair (3).
- Mark the rest of clauses that limmit somehow the liability of the service provider as potentially unfair (2).
- Mark only the minimun amount of text necessary to understand the clause.
"""

ch_system_prompt = """
You are a lawyer and you are reviewing online Terms of Service in search of Unilateral Change clauses
Unilateral change clauses specifies the conditions under which the service provider could amend and modify the terms of service and/or the service itself.
Follow this instructions carefully:
- Identify all unilateral change clauses in the given ToS.
- Mark all clauses of this kind as potentially unfair (2).
"""

ter_system_prompt = """
You are a lawyer and you are reviewing online Terms of Service in search of Termination clauses.
Termination clauses gives provider the right to suspend and/or terminate the service and/or the contract, and sometimes details the circumstances under which the provider claims to have a right to do so.
Follow this instructions carefully:
- Identify all termination clauses in the given ToS.
- Mark all clauses that do not specify reasons for termination as clearly unfair (2).
- Mark all clauses stipulating that the service provider may suspend or terminate the service at any time for any or no reasons and/or without notice as clearly unfair (3).
- Mark only the minimun amount of text necessary to understand the clause.
"""

use_system_prompt = """
You are a lawyer and you are reviewing online Terms of Service in search of Contract by Using clauses.
Contract by using clauses stipulates that the consumer is bound by the terms of use of a specifc service, simply by using the service, without even being required to mark that he or she has read and accepted them.
Follow this instructions carefully:
- Identify all contract by using clauses in the given ToS.
- Mark all clauses of this kind as potentially unfair (2).
- Mark only the minimun amount of text necessary to understand the clause.
"""

cr2_system_prompt = """
You are a lawyer and you are reviewing online Terms of Service in search of Content Removal clauses.
Content removal clauses gives the provider a right to modify/delete user’s content, including in-app purchases, and sometimes specifes the conditions under which the service provider may do so.
Follow this instructions carefully:
- Identify all content removal clauses in the given ToS.
- Mark all clauses that indicate conditions for content removal as potentially unfair (2).
- Mark all clauses that stipulate that the service provider may remove content in his full discretion, and/or at any time for any or no reasons and/or without notice nor possibility to retrieve the content as clearly unfair (3).
- Mark only the minimun amount of text necessary to understand the clause.
"""

a_system_prompt = """
You are a lawyer and you are reviewing online Terms of Service in search of Arbitration clauses.
Arbitration clauses requires or allows the parties to resolve their disputes through an arbitration process, before the case could go to court.
Follow this instructions carefully:
- Identify all arbitration clauses in the given ToS.
- Mark clauses defining arbitration as fully optional as clearly fair (1).
- Mark clauses not defining clearly if the arbitration is optional or mandatory as potentially unfair (2).
- Mark clauses stipulating that the arbitration should take place in a state other than the state of consumer’s residence and/or be based not on law but on arbiter’s discretion as clearly unfair (3).
- Mark only the minimun amount of text necessary to understand the clause.
"""

In [140]:
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", ltd_system_prompt),
    ("human", '"""{input}"""')
])

chain = create_structured_output_chain(UnfairClauses, llm, prompt, verbose=False)

In [141]:
result = chain.invoke(github_tos)

In [142]:
for clause in result["function"].clauses:
    print(clause.unfairness)
    print(clause.text)

2
In no event will either party be liable for indirect, incidental, special, punitive, or consequential damages, or loss of use, loss of profits, or interruption of business; however, caused or on any theory of liability.
1
No limitation or exclusions will apply to liability arising out of either party’s (1) confidentiality obligations unrelated to Customer Content; (2) defense obligations above; or (3) violation of the other party’s intellectual property rights.
